## Scrapy基本使用
　　参考：[官方文档](https://scrapy-chs.readthedocs.io/zh_CN/latest/intro/tutorial.html)<br>
　　　　　[]()<br>

　　初步了解使用Scrapy框架创建项目，并提取数据：
>创建一个Scrapy项目<br>
定义提取的Item<br>
编写爬取网站的 spider 并提取 Item<br>
编写 Item Pipeline 来存储提取到的Item(即数据)<br>

### 一、创建项目
　　在开始爬取之前，您必须创建一个新的Scrapy项目。 进入您打算存储代码的目录中，运行下列命令:<br>
>scrapy startproject tutorial<br>

该命令将会创建包含下列内容的 tutorial 目录:<br>
>tutorial/<br>
    scrapy.cfg　　　　　　　#  项目的主配置信息。（真正爬虫相关的配置信息在settings.py文件中）<br>
    tutorial/ 　　　　　　　# 该项目的python代码模块。之后您将在此加入代码。<br>
        __init__.py<br>
        items.py　　　　　# 设置数据存储模板，用于结构化数据，如Django的Model<br>
        pipelines.py　　　# 项目中的数据处理管道pipelines文件.<br>
        settings.py　　　　# 爬虫项目的设置文件.<br> 
        spiders/　　　　　# 放置spider代码的目录.<br>
            __init__.py<br>
            ...<br>

### 二、创建项目爬虫
　　为了创建一个Spider，您必须继承 scrapy.Spider 类 或其子类， 且定义以下三个属性:<br>
>name: 用于区别Spider。 该名字必须是唯一的，您不可以为不同的Spider设定相同的名字。<br>
start_urls: 包含了Spider在启动时进行爬取的url列表。 因此，第一个被获取到的页面将是其中之一。 后续的URL则从初始的URL获取到的数据中提取。<br>
parse() 是spider的一个方法。 被调用时，每个初始URL完成下载后生成的 Response 对象将会作为唯一的参数传递给该函数。 该方法负责解析返回的数据(response data)，提取数据(生成item)以及生成需要进一步处理的URL的 Request 对象。<br>
  
　　进入创建的Scrapy项目文件夹内，执行如下命令，创建指定的爬虫Spider.<br>
>rapy genspider qiubai www.qiushibaike.com<br>

　　执行成功后，就可以在项目的spiders目录下找到新生成的爬虫文件了。<br>

　　1）创建爬虫程序语法<br>
>rapy genspider 爬虫名称 起始URL或网站域名<br>

　　注意：URL或域名不可加引号。<br>
　　2）创建的爬虫文件内容模板<br>

>\# -\*- coding: utf-8 -\*-<br>
import scrapy<br>
<br>
class QiubaiSpider(scrapy.Spider):  # Spider是所有爬虫的父类<br>
    name = 'qiubai'   # 爬虫文件的名称：通过爬虫文件的名称可以指定定位到某一个具体的爬虫文件<br>
    allowed_domains = ['www.qiushibaike.com']   # 允许的域名：只爬取指定域名下的页面数据<br>
    start_urls = ['http://www.qiushibaike.com/']   # 起始url：当前工程将要爬取页面对应的url<br>
<br>
    def parse(self, response):<br>
        """<br>
        解析方法：对获取的页面数据进行指定内容解析<br>
        :param response: 根据起始url列表发起请求，请求成功返回的响应对象<br>
        :return:<br>
        """<br>
        pass<br>

### 三、定义项目采集的Item
　　Item 是保存爬取到的数据的容器；其使用方法和python字典类似， 并且提供了额外保护机制来避免拼写错误导致的未定义字段错误。<br>
　　类似在ORM中做的一样，您可以通过创建一个 scrapy.Item 类， 并且定义类型为 scrapy.Field 的类属性来定义一个Item。 (如果不了解ORM, 不用担心，您会发现这个步骤非常简单)<br>
　　首先根据需要从dmoz.org获取到的数据对item进行建模。 我们需要从dmoz中获取名字，url，以及网站的描述。 对此，在item中定义相应的字段。编辑 tutorial 目录中的 items.py 文件:<br>
>import scrapy<br>
<br>
class DmozItem(scrapy.Item):<br>
    title = scrapy.Field()<br>
    link = scrapy.Field()<br>
    desc = scrapy.Field()<br>
    
一开始这看起来可能有点复杂，但是通过定义item， 您可以很方便的使用Scrapy的其他方法。而这些方法需要知道您的item的定义。<br>

### 四、启动项目爬虫
　　进入项目的根目录，执行下列命令启动spider:<br>
>scrapy crawl qiubai<br>

　　crawl qiubai 启动用于爬取 www.qiushibaike.com 的spider 。<br>

　　刚才发生了什么？<br>
Scrapy为Spider的 start_urls 属性中的每个URL创建了 scrapy.Request 对象，并将 parse 方法作为回调函数(callback)赋值给了Request。Request对象经过调度，执行生成 scrapy.http.Response 对象并送回给spider parse() 方法。<br>
<br>
### 五、项目爬虫解析提取Item
　　Scrapy使用了一种基于 XPath 和 CSS 表达式机制: Scrapy Selectors 。 为了配合XPath，Scrapy除了提供了 Selector 之外，还提供了方法来避免每次从response中提取数据时生成selector的麻烦。<br>
　　Selector有四个基本的方法(点击相应的方法可以看到详细的API文档):<br>

|方法|注释|
|:---|:---|
|xpath(): |传入xpath表达式，返回该表达式所对应的所有节点的selector list列表 。|
|css(): |传入CSS表达式，返回该表达式所对应的所有节点的selector list列表。|
|extract(): |序列化该节点为unicode字符串并返回list。|
|re(): |根据传入的正则表达式对数据进行提取，返回unicode字符串list列表。|

　　在Shell中尝试Selector选择器:为了介绍Selector的使用方法，接下来我们将要使用内置的 Scrapy shell 。Scrapy Shell需要您预装好IPython(一个扩展的Python终端)。<br>
　　进入项目的根目录，执行下列命令来启动shell:<br>
>scrapy shell "http://www.dmoz.org/Computers/Programming/Languages/Python/Books/"<br>

　　注意:当您在终端运行Scrapy时，请一定记得给url地址加上引号，否则包含参数的url(例如 & 字符)会导致Scrapy运行失败。<br>
　　当shell载入后，您将得到一个包含response数据的本地 response 变量。输入 response.body 将输出response的包体，输出 response.headers 可以看到response的包头。<br>
　　更为重要的是，当输入 response.selector 时，您将获取到一个可以用于查询返回数据的selector(选择器)，以及映射到 response.selector.xpath() 、 response.selector.css() 的快捷方法(shortcut): response.xpath()和response.css()。同时，shell根据response提前初始化了变量 sel 。该selector根据response的类型自动选择最合适的分析规则(XML vs HTML)。<br>

### 六、修改settings.py配置文件
\# Crawl responsibly by identifying yourself (and your website) on the user-agent<br>
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36' # 伪装请求载体身份<br>

\# Obey robots.txt rules<br>
ROBOTSTXT_OBEY = False   # 不遵从门户网站robots协议，避免某些信息爬取不到<br>
注意：<br>
　　（1）取消USER_AGENT注释，这里给它添加火狐浏览器身份标识，以伪装请求载体身份。<br>
　　（2）将ROBOTSTXT_BOE修改为False，即不遵守门户网站的robots协议，避免某些信息爬取不到。（这个可视情况决定是否遵守）<br>

### 七、保存爬取到的数据
>scrapy crawl dmoz -o items.json

该命令将启动爬虫、并采用 JSON 格式对爬取的数据进行序列化存储，生成 items.json 文件。<br>









